<a href="https://colab.research.google.com/github/matlogica/AADC-Python/blob/main/QuantLib/03-OIS-QuantLib-example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Example to show how to use the AADC with QuantLib library in Python. Here we create QL Zero curves and price single OIS swap. 
We also show how to use the AADC to compute the sensitivities of the swap to the zero rates.
The example is based on the QuantLib example available at https://www.quantlib.org/reference/quantlib/instruments/ois.html
At the end of the example we compare the sensitivities computed by the AADC with the sensitivities computed by bumping the zero rates.


In [30]:
import sys
!pip install https://matlogica.com/DemoReleases/aadcquantlib-1.7.5.27-cp3{sys.version_info.minor}-cp3{sys.version_info.minor}-linux_x86_64.whl

  Using cached https://matlogica.com/DemoReleases/aadcquantlib-1.7.5.27-cp311-cp311-linux_x86_64.whl (38.0 MB)


In [31]:
import aadc
import aadc.quantlib as ql
import numpy as np

Initialize QuantLib as normal

In [32]:
today = ql.Date(19, ql.October, 2020)
ql.Settings().evaluationDate = today

In [33]:
# create the yield curve zero rates using native double arrays
dates = [    ql.Date(19,10,2020),    ql.Date(19,11,2020),    ql.Date(19, 1,2021),    ql.Date(19, 4,2021),    ql.Date(19,10,2021),    ql.Date(19, 4,2022),    ql.Date(19,10,2022),    ql.Date(19,10,2023),    ql.Date(19,10,2025),    ql.Date(19,10,2030),    ql.Date(19,10,2035),    ql.Date(19,10,2040),]

forecast_rates = [    -0.004,    -0.002,    0.001,    0.005,    0.009,    0.010,    0.010,    0.012,    0.017,    0.019,    0.028,    0.032]


In [34]:
discount_spread = - 0.001

In [35]:
# Define function to price single OIS swap. Note that discount_rates are computed on Python side, but pricing is done on C++ side.
# We calculate risk w.r.t forecast rates and discount spread. AAD tape is used to compute sensitivities on python and C++ side.

def PriceOIS(dates, forecast_rates, discount_spread):
    forecast_curve = ql.ZeroCurve(dates, forecast_rates, ql.Actual365Fixed())
    
    discount_rates = [ r + discount_spread for r in forecast_rates ]
    discount_curve = ql.ZeroCurve(dates, discount_rates, ql.Actual365Fixed())

    forecast_handle = ql.YieldTermStructureHandle(forecast_curve)
    discount_handle = ql.YieldTermStructureHandle(discount_curve)

    swap = ql.MakeOIS(swapTenor=ql.Period(5, ql.Years),
                    overnightIndex=ql.Eonia(forecast_handle),
                    fixedRate=0.002)

    swapEngine = ql.DiscountingSwapEngine(discount_handle)
    swap.setPricingEngine(swapEngine)

    return swap.NPV()

In [36]:
# Price the swap using regular call to QuantLib with native double types
print(PriceOIS(dates, forecast_rates, discount_spread))

idouble(7.22e-02)


In [37]:
# Create object to hold AADC Kernel recording
funcs = aadc.Functions()

In [38]:
# Start recording. All aadc.idouble() operations will be recorded
funcs.start_recording()

You are using evaluation version of AADC. Expire date is 20240901


In [39]:
# convert each element in rates and discount_spread to idouble
forecast_rates = [aadc.idouble(r) for r in forecast_rates]
discount_spread = aadc.idouble(discount_spread)
forecast_rates

[idouble(-4.00e-03),
 idouble(-2.00e-03),
 idouble(1.00e-03),
 idouble(5.00e-03),
 idouble(9.00e-03),
 idouble(1.00e-02),
 idouble(1.00e-02),
 idouble(1.20e-02),
 idouble(1.70e-02),
 idouble(1.90e-02),
 idouble(2.80e-02),
 idouble(3.20e-02)]

In [40]:
# mark each idouble as input and save the reference id
forecast_ratesArgs = [r.mark_as_input() for r in forecast_rates]
discount_spreadArg = discount_spread.mark_as_input()
forecast_ratesArgs

[Arg(6),
 Arg(7),
 Arg(8),
 Arg(9),
 Arg(10),
 Arg(11),
 Arg(12),
 Arg(13),
 Arg(14),
 Arg(15),
 Arg(16),
 Arg(17)]

In [41]:
# Call the PriceOIS function with the idouble arguments and record the operations
swap_NPV = PriceOIS(dates, forecast_rates, discount_spread)

In [42]:
# Note that swap_NPV is an aadc.idouble() object. [rv] - random variable [adj] - adjoint flag
print(swap_NPV)

idouble([AAD[rv] [adj] :1492,7.22e-02])


In [43]:
# Mark the swap_NPV as output and save the reference
swapNPVRes = swap_NPV.mark_as_output()
swapNPVRes

Res(1492)

In [44]:
# Stop recording
funcs.stop_recording()

In [45]:
# Check if recording is safe to use for arbitrary inputs
# This will print the list of active to passive extract locations
# To use the recording for arbitrary inputs, the active to passive extract locations should be 0
funcs.print_passive_extract_locations()

Number active to passive conversions: 0 while recording Python


'Number active to passive conversions: 0 while recording Python\n'

In [46]:
# swap_NPV is still an aadc.idouble() object after recording
print(swap_NPV)

idouble(7.22e-02)


In [47]:
# But it can be safely converted to a native double
print(swap_NPV+0.0)

0.07217327877335217


In [48]:
# Create new input zero rates and discount spread
# Set vol to 0 to avoid random perturbations and compare bump sensitivities with AADC sensitivities
vol=0.01
num_scenarios=10000

In [49]:
# Basic perturbations to the zero rates and discount spread
inputs = {}
for rArg, r in zip(forecast_ratesArgs, forecast_rates):
    inputs[rArg] = float(r) * np.random.normal(1, vol, num_scenarios) 
inputs[discount_spreadArg] = discount_spread * np.random.normal(1, vol, num_scenarios)
inputs[discount_spreadArg]

array([-0.00101741, -0.00099486, -0.00100568, ..., -0.0009908 ,
       -0.00100606, -0.00097302])

In [50]:
# Request to AADC evaluation.
# We want swapNPV output and the sensitivities of swapNPV to the zero rates and discount spread
request = {swapNPVRes:forecast_ratesArgs + [ discount_spreadArg ]}

In [51]:
# Run AADC Kernel for array inputs using 4 CPU threads and avx2
Res = aadc.evaluate(funcs, request, inputs, aadc.ThreadPool(4))

In [52]:
# AADC returns vector of results for each scenario
print("Result", Res[0][swapNPVRes])

Result [0.07316674 0.07145473 0.07074005 ... 0.07177836 0.0712956  0.0722611 ]


In [53]:
# Bucketed sensitivities of swapNPV to the zero rates
print("Forecast rates risk : ")
for rArg in forecast_ratesArgs:
    print("dNPV/dR", np.average(Res[1][swapNPVRes][rArg]))

Forecast rates risk : 
dNPV/dR -0.005131204700891976
dNPV/dR -0.00035387618626841225
dNPV/dR 0.0
dNPV/dR 0.0
dNPV/dR 0.0010133171488007587
dNPV/dR 1.125907943111949e-05
dNPV/dR 0.0020013645317468213
dNPV/dR 0.004953017979009238
dNPV/dR 4.628746115605565
dNPV/dR 0.005073220061847135
dNPV/dR 0.0
dNPV/dR 0.0


In [54]:
# Sensitivity of swapNPV to the discount spread
print("Discount spread risk : ")
print("dNPV/dR", (Res[1][swapNPVRes][discount_spreadArg]))

Discount spread risk : 
dNPV/dR [-0.26444274 -0.25718616 -0.2536319  ... -0.25782956 -0.25578638
 -0.26112541]


In [55]:
# Compare AADC sensitivities with sensitivities computed by bumping the zero rates
# For perfect match make sure to set vol=0
print("Check with bumping")
swap_NPV = PriceOIS(dates, forecast_rates, discount_spread)
print("Forecast rates risk : ")
for i in range(len(forecast_rates)):
    forecast_rates[i] += 0.0001
    swap_NPV_up = PriceOIS(dates, forecast_rates, discount_spread)
    forecast_rates[i] -= 0.0002
    swap_NPV_down = PriceOIS(dates, forecast_rates, discount_spread)
    forecast_rates[i] += 0.0001
    AAD_risk = np.average(Res[1][swapNPVRes][forecast_ratesArgs[i]])
    FD_risk =(swap_NPV_up - swap_NPV_down) / 0.0002 
    print(i, FD_risk, AAD_risk, FD_risk - AAD_risk)

Check with bumping
Forecast rates risk : 
0 -0.0051312044734713735 -0.005131204700891976 2.2742060231534245e-10
1 -0.00035387617086657386 -0.00035387618626841225 1.5401838387562455e-11
2 0.0 0.0 0.0
3 0.0 0.0 0.0
4 0.0010133604465972779 0.0010133171488007587 4.329779651913307e-08
5 1.1259560089849074e-05 1.125907943111949e-05 4.806587295843841e-10
6 0.0020014580717070407 0.0020013645317468213 9.353996021938676e-08
7 0.004953248990344217 0.004953017979009238 2.3101133497880483e-07
8 4.628780212303298 4.628746115605565 3.409669773368762e-05
9 0.005073257131407716 0.005073220061847135 3.7069560580860195e-08
10 0.0 0.0 0.0
11 0.0 0.0 0.0


In [56]:
# Sensitivity of swapNPV to the discount spread
# For perfect match make sure to set vol=0
print("Discount spread risk : ")
discount_spread += 0.0001
swap_NPV_up = PriceOIS(dates, forecast_rates, discount_spread)
discount_spread -= 0.0002
swap_NPV_down = PriceOIS(dates, forecast_rates, discount_spread)
discount_spread += 0.0001
AAD_risk = np.average(Res[1][swapNPVRes][discount_spreadArg])
FD_risk =(swap_NPV_up - swap_NPV_down) / 0.0002
print(FD_risk, AAD_risk, FD_risk - AAD_risk)

Discount spread risk : 
-0.25997967206814465 -0.2600059759712291 2.6303903084445857e-05
